In [1]:
import numpy as np 
import pandas as pd
import re

pd.set_option('display.max_columns', 24)

In [2]:
# helper function to clean columns
def remove_crap(string_):
    """
    Accepts a string as input parameter and returns a 'cleaned' string (only includes #s and letters)

    input parameter:
    ---------------
    string

    output:
    ------
    string
     """

    # remove brackets and single quotes using regex substitution
    output = re.sub("[^a-zA-Z0-9 ]", "", string_)

    # return cleaned string
    return output

In [3]:
# helper function to isolate music genres that include the word 'rock'
def get_rock(genre_items):
    """
    Accepts a genre items list (i.e. ['canadian', 'progressive', 'folk rock']) and 
    returns 0 if 'rock' is not included in genre_items, otherwise returns the original list
    
    input parameter:
    ---------------
    list of strings
    
    output:
    ------
    list of strings or zero
    """
    if 'rock' in genre_items:
        return genre_items
    else:
        return 0

In [15]:
def wrangle():
    
    # open .csv files and turn into dataframes
    df1 = pd.read_csv("./data/artists.csv")
    df6 = pd.read_csv("./data/tracks.csv")
    
    # rename column to assist with merging of dataframes
    df1.rename({'id': 'id_artists'}, axis=1, inplace=True)
    
    # clean df6.id_artists for easier merging of dataframes
    df6['id_artists'] = df6['id_artists'].apply(remove_crap)

    # merge df1 and df6 
    m_df = df1.merge(df6, how='inner', on ='id_artists')
    
    # prepare 'genres' column to isolate out instances that include the word 'rock'
    m_df['genres'] = m_df['genres'].apply(get_rock)
    
    # create dataframe of only genres that contain the word 'rock'
    rock_df = m_df[m_df['genres'] != 0]
    
    # create a copy of rock_df
    clean_rock_df = rock_df.copy()
    
    # columns to drop
    cols = ['id_artists', 'followers', 'popularity_x', 'id', 'popularity_y', 'artists']
    
    # drop unnecessary columns
    clean_rock_df.drop(columns=cols, inplace=True)
    
    # clean to 'genres column' to include only a-zA-Z0-9
    clean_rock_df['genres'] = clean_rock_df['genres'].apply(remove_crap)
    
    # rename columns with more appropriate name spaces
    clean_rock_df.rename({'name_x': 'artists'}, axis=1, inplace=True)
    clean_rock_df.rename({'name_y': 'song'}, axis=1, inplace=True)
    
    # convert 'release_date' to datetime objects
    clean_rock_df['release_date'] = pd.to_datetime(clean_rock_df['release_date'], infer_datetime_format=True)
    
    # keep only the year of 'release_date'
    clean_rock_df['release_date'] = clean_rock_df['release_date'].dt.year
    
    # drop the 1st 30,000 entries (to further shrink the dataset to < 25MB; all drops are non-U.S. artists)
    clean_rock_df = clean_rock_df[30000:]
    
    # reset index
    clean_rock_df.reset_index(inplace=True)
    
    # drop old index column
    clean_rock_df.drop(columns=['index'], inplace=True)
    
    return clean_rock_df

In [16]:
df = wrangle()

In [17]:
df.head()

,genres,artists,song,duration_ms,explicit,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,musica tropical guatemalteca rock chapin rock ...,Bohemia Suburbana,Retiro,261000,0,2001,0.433,0.499,4,-7.664,1,0.0277,0.4670,0.285000,0.126,0.204,132.986,3
1,musica tropical guatemalteca rock chapin rock ...,Bohemia Suburbana,Tengo Que Llegar,233693,0,2015,0.400,0.682,4,-6.609,0,0.0318,0.1500,0.336000,0.246,0.322,147.988,4
2,musica tropical guatemalteca rock chapin rock ...,Bohemia Suburbana,Pero Nadie,258027,0,2015,0.539,0.753,0,-6.430,0,0.0375,0.0691,0.000706,0.182,0.361,124.041,4
3,adoracao brazilian gospel louvor rock gospel b...,Daniela Araújo,Guia-Me,237820,0,2011,0.523,0.297,1,-9.878,1,0.0290,0.6600,0.002390,0.129,0.228,135.975,4
4,ccm christian alternative rock christian music...,Audio Adrenaline,Big House,211667,0,1993,0.769,0.841,9,-8.006,1,0.0748,0.0538,0.000024,0.067,0.898,112.014,4


In [23]:
tester = df[df['artists'] == 'Van Halen']

In [24]:
tester.head(10)

,genres,artists,song,duration_ms,explicit,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
57409,album rock classic rock hard rock metal rock,Van Halen,Runnin' with the Devil - 2015 Remaster,214960,0,1978,0.472,0.900,8,-6.406,0,0.0582,0.1480,0.000297,0.340,0.420,95.826,4
57410,album rock classic rock hard rock metal rock,Van Halen,Ain't Talkin' 'Bout Love - 2015 Remaster,227800,0,1978,0.518,0.938,8,-5.964,0,0.0416,0.0210,0.000305,0.288,0.710,139.138,4
57411,album rock classic rock hard rock metal rock,Van Halen,You Really Got Me - 2015 Remaster,156147,0,1978,0.454,0.946,1,-5.978,0,0.1300,0.0223,0.000000,0.173,0.647,138.180,4
57412,album rock classic rock hard rock metal rock,Van Halen,Eruption - 2015 Remaster,102080,0,1978,0.225,0.689,8,-9.120,1,0.0555,0.4690,0.813000,0.337,0.500,93.798,4
57413,album rock classic rock hard rock metal rock,Van Halen,Jamie's Cryin' - 2015 Remaster,209587,0,1978,0.556,0.931,8,-6.125,1,0.0429,0.0970,0.001820,0.231,0.700,128.001,4
57414,album rock classic rock hard rock metal rock,Van Halen,Ice Cream Man - 2015 Remaster,198373,0,1978,0.402,0.759,10,-7.685,0,0.1340,0.0698,0.000000,0.193,0.579,88.085,4
57415,album rock classic rock hard rock metal rock,Van Halen,Feel Your Love Tonight - 2015 Remaster,220960,0,1978,0.460,0.862,8,-5.225,1,0.0594,0.0661,0.000000,0.188,0.724,135.464,4
57416,album rock classic rock hard rock metal rock,Van Halen,I'm the One - 2015 Remaster,224600,0,1978,0.310,0.980,1,-5.691,1,0.1970,0.0152,0.000001,0.192,0.237,114.041,4
57417,album rock classic rock hard rock metal rock,Van Halen,Little Dreamer - 2015 Remaster,201760,0,1978,0.382,0.811,11,-6.191,0,0.0361,0.0240,0.000001,0.261,0.733,179.149,4
57418,album rock classic rock hard rock metal rock,Van Halen,Atomic Punk - 2015 Remaster,179613,0,1978,0.416,0.993,6,-6.354,1,0.1850,0.0131,0.000633,0.589,0.263,100.364,4


In [25]:
df.to_csv("./spotify_rock.csv")

In [26]:
df.shape

(114157, 18)